<a href="https://colab.research.google.com/github/AkibCoding/Sentiment-Analysis-Using-Bert-Base-on-SST2-dataset/blob/main/SentimentAnalysisBertBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.0 MB/s eta 0:00:00


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.2 MB/s eta 0:00:00


In [5]:
!pip install torch transformers datasets tqdm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader, random_split
from transformers.optimization import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm

# Load the dataset
dataset = load_dataset("glue", "sst2")

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input sentences
def tokenize(batch):
    return tokenizer(batch['sentence'], padding='longest', truncation=True)

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize, batched=True)

# Set the random seed for reproducibility
torch.manual_seed(42)

# Split the dataset into training and validation sets
train_dataset, eval_dataset = tokenized_dataset['train'], tokenized_dataset['validation']
num_train_examples = len(train_dataset)




  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
# Define the model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataset) // 8
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the batch size and number of training epochs
batch_size = 8
num_epochs = 5

# Custom collate function to handle variable-length sequences
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    labels = [item['label'] for item in batch]

    max_seq_len = max(len(seq) for seq in input_ids)
    padded_input_ids = [seq + [tokenizer.pad_token_id] * (max_seq_len - len(seq)) for seq in input_ids]
    padded_attention_masks = [seq + [0] * (max_seq_len - len(seq)) for seq in attention_masks]

    return {
        'input_ids': torch.tensor(padded_input_ids).to(device),
        'attention_mask': torch.tensor(padded_attention_masks).to(device),
        'label': torch.tensor(labels).to(device)
    }



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
# Create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0

    # Iterate over batches
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        inputs = batch['input_ids']
        masks = batch['attention_mask']
        labels = batch['label']

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_mask=masks)
        logits = outputs.logits

        # Compute loss and backpropagation
        loss = loss_fn(logits, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

        # Compute training accuracy
        _, predicted_labels = torch.max(logits, 1)
        correct_predictions += torch.sum(predicted_labels == labels).item()


Epoch 5: 100%|██████████| 8419/8419 [15:41<00:00,  8.94it/s]


In [19]:

    # Compute validation accuracy
    model.eval()
    with torch.no_grad():
        correct_val_predictions = 0
        for batch in eval_loader:
            inputs = batch['input_ids']
            masks = batch['attention_mask']
            labels = batch['label']

            outputs = model(inputs, attention_mask=masks)
            logits = outputs.logits

            _, predicted_labels = torch.max(logits, 1)
            correct_val_predictions += torch.sum(predicted_labels == labels).item()

    avg_loss = total_loss / len(train_loader)
    train_accuracy = correct_predictions / num_train_examples
    val_accuracy = correct_val_predictions / len(eval_dataset)

    print(f"Epoch {epoch + 1}:")
    print(f"Training Loss: {avg_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print('-' * 50)

Epoch 5:
Training Loss: 0.0917, Training Accuracy: 0.9697
Validation Accuracy: 0.9220
--------------------------------------------------
